# Lab 06: Recurrent Neural Network (RNN)

Trong bài thực hành này:
- Cài đặt 1 mạng RNN cơ bản LSTM
- Sử dụng Word Embedding GLOVE của Stanford
- Chạy trên data spam detection

Reference:
- Glove: https://github.com/stanfordnlp/GloVe
- LSTM: Long Short-Term Memory layer - Hochreiter 1997.

Đọc thêm:
- LSTM: https://colah.github.io/posts/2015-08-Understanding-LSTMs/

## Tiền xử lý dữ liệu

Chúng ta cần tách câu thành từng từ trước.

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
path='gdrive/My Drive/DATASCIENCE/MayHocNangCao/Lab06/'

In [3]:
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')

df = pd.read_csv(path+"spam_detection.csv")
df.head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


,Text,Label,y
0,"Go until jurong point, crazy.. Available only ...",ham,0
1,Ok lar... Joking wif u oni...,ham,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam,1
3,U dun say so early hor... U c already then say...,ham,0
4,"Nah I don't think he goes to usf, he lives aro...",ham,0


In [4]:
texts = df["Text"].to_list()
texts = [text.lower() for text in texts]            # chuyển các đoạn text thành chữ thường (word embedding chỉ cho chữ thường)
tokenized_texts = [nltk.tokenize.word_tokenize(text) for text in texts]   # tách câu thành một list các từ

print(tokenized_texts[1])

['ok', 'lar', '...', 'joking', 'wif', 'u', 'oni', '...']


## Load embedding từ file

Pretrained Embeddings từ Glove-Stanford đã được rút gọn cho bài tập này và lưu thành file glove_embedding.txt.

In [0]:
## không cần hiểu đống này lắm đâu
import io
import numpy as np
def load_word_embeddings(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    vocab, matrix = [], []
    i=0
    for line in fin:
        tokens = line.rstrip().split(' ')
        vocab.append(tokens[0])
        matrix.append(list(map(float, tokens[1:])))
    return vocab, np.asarray(matrix)

In [0]:
vocab, matrix = load_word_embeddings(path+"glove_embedding.txt")

Sau khi đọc xong thì
- vocab: một danh sách các từ vực có trong embedding
- matrix: một ma trận, mỗi dòng là một embedding cho từ tương ứng trong vocab (xếp đúng thứ tự)

## Số hóa data

Để số hóa 1 từ (word) trong ngôn ngữ tự nhiên, người ta sẽ biểu diễn từ đó thành một vector (gọi là embedding). 2 bước trước ta đã tách các câu trong data thành từ riêng biệt, và load một bộ embedding có sẵn. Bây giờ ta chuyển từng từ trong data thành một mã số biểu thị vị trí của từ đó trong ma trận embedding.

Tuy nhiên, ta cần có vài mã số đặc biệt để giải quyết các vấn đề như: 
- từ không có trong embedding
- Độ dài các câu không giống nhau. Cơ bản, các thư viện deep learning tính toán nhanh dựa trên các kĩ thuật tính toán ma trận (tensor), nên để tính các câu có độ dài ngắn khác nhau, các câu ngắn cần được nối thêm bởi các mã đặc biệt để có cùng kích thước.



In [0]:
## Gán các mã
__PADDED_INDEX__ = 0    # mã dùng cho các vị trí chỉ có tính nối dài cho cùng kích thước
__UNKNOWN_WORD__ = 1    # mã cho những từ không có trong embedding

In [0]:
# Tạo một dictionary, có nhiệm vụ là một ánh xạ từ ảnh sang mã số, mã số được bắt đầu từ 2 vì số 0 và 1 được dành cho trường hợp đặc biệt
word_to_index = {word: index+2 for index, word in enumerate(vocab)}


In [9]:
# Do do mã số được bắt đầu từ 2, nên cần thêm 2 vector vào đàu ma trận
embedding_matrix = np.pad(matrix, [[2,0],[0,0]], mode='constant', constant_values =0.0)
print(embedding_matrix)

# Khi đó, __PADDED_INDEX__ dùng dòng đầu tiên của  embedding_matrix
# __UNKNOWN_WORD__ dùng dòng thứ hai của embedding_matrix 

[[ 0.         0.         0.        ...  0.         0.         0.       ]
 [ 0.         0.         0.        ...  0.         0.         0.       ]
 [ 0.18378   -0.12123   -0.11987   ... -0.039038   0.18274    0.14654  ]
 ...
 [ 0.18754   -0.040832   0.19611   ...  0.079996   0.016479   0.17797  ]
 [-0.1167     0.0073071 -0.19401   ...  0.21549    0.015527   0.14948  ]
 [ 0.075198  -0.097452  -0.14505   ...  0.23842   -0.39966    0.14454  ]]


In [10]:
## Bây giờ ta sẽ chuyển data spam dection thành các mã số
import tensorflow as tf

X = []
for text in tokenized_texts:
    cur_text_indices = []
    for word in text:
        if word in word_to_index:
            cur_text_indices.append(word_to_index[word])    ## map từ word sang index
        else:
            cur_text_indices.append(__UNKNOWN_WORD__)       ## gán unknown cho từ không có trong bộ glove
    X.append(cur_text_indices)

## pad data cho có cùng chiều dài
X = tf.keras.preprocessing.sequence.pad_sequences(sequences=X,       # sequences: list các câu có độ dài không bằng nhau
                                                  padding='post')    # vị trí pad là 'pre' (trước) hoặc 'post' (sau)

y = df['y'].values   ## Label của bài toán, 0 là không phải spam, 1 là spam

In [0]:
## Chuyển label của bài toán

int2label = {0: "spam", 1: "ham"}

In [0]:
## Chia data
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size= 0.2, random_state =0)

## RNN trong tensorflow

In [13]:
from tensorflow.keras.layers import Input, Embedding, LSTM, Bidirectional, Dense
from tensorflow.keras.models import Model

inputs = Input(shape=(None,))                   ## None biểu thị kích thước không xác định của câu

embed = Embedding(input_dim=embedding_matrix.shape[0],   ## Khai báo kích thước của vocab
                 output_dim=embedding_matrix.shape[1],   ## Khai báo kích thước của embedding
                  embeddings_initializer = tf.keras.initializers.Constant(value=embedding_matrix),  ## Khởi tạo cho embedding bằng ma trận có sẵn
                  trainable=False,                       ## Không cần thiết train embedding
                 mask_zero=True)(inputs)                 ## zero_mask: những vị trí có giá trị 0 không được tính toán, vì đó là giá trị thêm vào cho đủ độ dài mà thôi
                                                         ##  (__PADDED_INDEX__ gán bằng 0)

lstm = Bidirectional(LSTM(units=100,                          ## units: kích thước của hidden_state trong LSTM
            return_sequences=False))(embed)      ## return_sequences: LSTM trả về toàn bộ  hay là trả về hidden_state cuối cùng

dense = Dense(units=2, activation='softmax')(lstm)
model = Model(inputs=inputs,
              outputs=dense)

model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])
model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding (Embedding)        (None, None, 300)         2208600   
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               32080

In [14]:
# Checkpoint Callback
mc = tf.keras.callbacks.ModelCheckpoint(filepath="lstm_spam.h5", 
                                     monitor='val_loss',
                                     mode='min', 
                                     verbose=0, 
                                     save_best_only=True)
# Train
model.fit(X_train, y_train, 
          validation_data=(X_valid, y_valid),
         epochs=10,
         callbacks=[mc])

model.load_weights("lstm_spam.h5")
_, val_acc = model.evaluate(X_valid, y_valid)
print("Accuracy on valid: ", val_acc)

Train on 4459 samples, validate on 1115 samples
Epoch 1/10
4459/4459 [==============================] - 108s 24ms/sample - loss: 0.1160 - acc: 0.9617 - val_loss: 0.0617 - val_acc: 0.9794
Epoch 2/10
4459/4459 [==============================] - 107s 24ms/sample - loss: 0.0378 - acc: 0.9879 - val_loss: 0.0605 - val_acc: 0.9794
Epoch 3/10
4459/4459 [==============================] - 107s 24ms/sample - loss: 0.0243 - acc: 0.9935 - val_loss: 0.0418 - val_acc: 0.9865
Epoch 4/10
4459/4459 [==============================] - 106s 24ms/sample - loss: 0.0159 - acc: 0.9962 - val_loss: 0.0512 - val_acc: 0.9794
Epoch 5/10
4459/4459 [==============================] - 108s 24ms/sample - loss: 0.0112 - acc: 0.9969 - val_loss: 0.0437 - val_acc: 0.9865
Epoch 6/10
4459/4459 [==============================] - 109s 24ms/sample - loss: 0.0033 - acc: 0.9993 - val_loss: 0.0492 - val_acc: 0.9865
Epoch 7/10
4459/4459 [==============================] - 108s 24ms/sample - loss: 0.0014 - acc: 0.9998 - val_loss: 0.05

In [0]:
def message_to_array(texts):
  texts = texts.lower().split(' ')            # chuyển các đoạn text thành chữ thường (word embedding chỉ cho chữ thường)
  #print(texts)
  tokenized_texts = [nltk.tokenize.word_tokenize(text) for text in texts]
  
  print(tokenized_texts)
 # text = text.lower().split(' ')

  cur_text_indices = []
  for text in tokenized_texts:
    for word in text:
        if word in word_to_index:
            cur_text_indices.append(word_to_index[word])    ## map từ word sang index
        else:
            cur_text_indices.append(__UNKNOWN_WORD__)       ## gán unknown cho từ không có trong bộ glove

           
  return cur_text_indices

In [0]:
def model_predict(text):
  model.load_weights("lstm_spam.h5")
  #model.summary()
  test = message_to_array(text)
  print(test)
  pred = model.predict(test)[0]
  
  return np.argmax(pred)

In [17]:
text = 'wanna ask something? just send me a mess'
print(model_predict(text))

[['wan', 'na'], ['ask'], ['something', '?'], ['just'], ['send'], ['me'], ['a'], ['mess']]
[4416, 2777, 639, 232, 39, 69, 645, 72, 8, 2688]
0


In [18]:
text = "Urgent! You have won our competition's prize!! Please call us now."
print(model_predict(text))

[['urgent', '!'], ['you'], ['have'], ['won'], ['our'], ['competition', "'s"], ['prize', '!', '!'], ['please'], ['call'], ['us'], ['now', '.']]
[3743, 37, 5669, 30, 1169, 64, 1453, 22, 2303, 37, 37, 223, 298, 107, 97, 4]
0


In [19]:
text = "Call me to get a free holiday now"
print(model_predict(text))

[['call'], ['me'], ['to'], ['get'], ['a'], ['free'], ['holiday'], ['now']]
[298, 72, 6, 80, 8, 128, 986, 97]
0


## Bài tập
- Nghiên cứu sử dụng tf.keras.layers.Bidirectional trong model. Lưu lý: cập nhật lên tensorflow 2.0 để Bidirectional chạy đúng mask_zero.
- Hãy viết một hàm
```python
def model_predict(text):
    
    return True/False
```
để đoán xem một đoạn text đưa vào có phải là tin nhắn spam không (các biến khác dùng global)
- Tự điều chỉnh và train model (chỉnh lại train, valid, tiền xử lý, dùng word_embedding,... nếu muốn) rồi đoán xem các câu sau có phải spam không:
    - "wanna ask something? just send me a mess"
    - "Urgent! You have won our competition's prize!! Please call us now."
    - "Call me to get a free holiday now"